<a href="https://colab.research.google.com/github/cda79/IAT360-LLM/blob/main/IAT360_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Required libraries

In [ ]:
!pip install transformers torch datasets

In [ ]:
!pip install transformers torch accelerate

##Data Pre-processing
Convert CSV to huggingface & gpt-2 format

**Import raw .csv data**

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("shakespeare_dataset.csv")

# display it just to test
print(df.head())
df.info()

**Format it for gpt-2**

SOURCE: [Source Text] TARGET: [Target Text] <|endoftext|>

The <|endoftext|> token is for GPT-2 to signal the end of a generated sequence.


In [ ]:
def format_translation_data(row):
  # whitespace cleanup
  shakespeare_text = row['shakespeare'].strip()
  modern_text = row['modern'].strip()
  formatted_text = f"SOURCE: {shakespeare_text} TARGET: {modern_text} <|endoftext|>"
  return formatted_text

# create new column for collated data to be tokenized
df['text_for_tokenization'] = df.apply(format_translation_data, axis=1)
# display it & check
print(df[['text_for_tokenization']].head())


Full display of the text for double checking since the previous output cuts it out

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

print(df[['text_for_tokenization']].to_string())

**Format for the Huggingface trainer**

In [ ]:
hf_dataset = Dataset.from_pandas(df)
print("\nConverted to Hugging Face Dataset format.")
print(hf_dataset)

## Tokenization

**Define the tokenizer & apply**

Default gpt-2 padding & parameters.

To understand how gpt-2 tokenizes the text, [see here.](https://blog.lukesalamone.com/posts/gpt2-tokenization/)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling

# gpt-2 tokenizer to make it machine readable
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# default padding token
tokenizer.pad_token = tokenizer.eos_token

# use the huggingface dataset we just converted
# hf_dataset = Dataset.from_pandas(df) << this was done in the previous step

def tokenize_function(examples):
    # uses the prepared 'text_for_tokenization' column
    return tokenizer(
        examples["text_for_tokenization"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

# apply tokenization
# done by mapping it to the dataset and then removing the columns without tokens
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True, remove_columns=['shakespeare', 'modern', 'text_for_tokenization'])

# Display example tokenized output with the first entry
print("\n dataset structure:")
print(tokenized_datasets)
# where the input is decomposed into a string of numbers or tokens - depending on gpt-2s vocabulary
print(tokenized_datasets[0]["input_ids"])



## Training

In [ ]:
train_dataset = tokenized_datasets

# since we only have a test csv i just made it train on all the data
# otherwise we'd split it like this

#tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)
# or for 80/20 split, where seed is the randomizer
# ... = tokenized_datasets.train_test_split(train_size=0.8, seed=40)

**Load the model and adjust the parameters**

Loading and deploying the gpt-2 model based off tokenized data: https://medium.com/@majd.farah08/generating-text-with-gpt2-in-under-10-lines-of-code-5725a38ea685

Documentation for data collating: https://huggingface.co/docs/transformers/main/main_classes/data_collator

... and what it does internally / the different types: https://towardsdatascience.com/data-collators-in-huggingface-a0c76db798d2/

In [ ]:
from transformers import TrainingArguments, Trainer
import torch

# load gpt-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# data collating allows us to use our csv datatables as the input for model
# the mlm specifies whether we should mask tokens or not - usually used for prediction
# when its set to false then the labels = inputs with the padding ignored
# so the whole thing is read AND it returns the labels which is important for our specific translation use
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# set up the trainer and its arguements
training_args = TrainingArguments(
    output_dir="./gpt2_shakespeare_translator",  # directory to save checkpoints
    num_train_epochs=50,                       # epoch size
    per_device_train_batch_size=4,             # batch size
    logging_steps=10,                          # log the stats of the training every 10 steps, creates a consistent loss chart
)

# initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)


##Ignore this code - for pre-made datasets

###Import pre-made dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("lanretto/shakespeare-vs-modern-dialogue")
dataset

In [ ]:
## For now, unused
# from datasets import load_dataset
# dataset = load_dataset("madha98/Shakespeare")
# dataset

###Data pre-processing - for training/test split

In [ ]:
#example print
dataset['train'][0]

In [ ]:
#shuffle and split the dataset into a smaller amount
#select only the first 100 and shuffle them up
dataset = dataset['train'].shuffle(seed=30).select(range(100))
dataset

In [ ]:
# create test and train dataset from this shuffled amount
# 80-20 split
dataset = dataset.train_test_split(train_size=0.8, seed=40)
dataset


### Load custom dataset files
For when we make our own, here is some temp code (not to be run)

In [ ]:
#Load files as dataset
data_files={"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Save it to our huggingface hub (not to be run)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

###Tokenization

So this is when we'd split our custom dataset into a third column that combines the features together into one line for training i think